# Bayesian Network Concepts
In this notebook, we will review some concepts about conditional probability tables, how to estimate them and how to use them. We will do this using the [pgmpy python suite](https://pgmpy.org/index.html). We will first import the set the paths and import pgmpy into the Jupyter environment. The notebook was tested with pgmpy version 0.1.19.

In [ ]:
import pgmpy

## Step 1. Create a small dataset 
This example is a toy example with just four variables. The are `fruit`, `tasty`, `size`, `color`. We will use the Python `pandas` package to create this dataset. This dataset has 14 samples.  

In [ ]:
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"],
                         'color': ["yellow","green","yellow","red","green","red","yellow",
                                  "red","yellow","green","green","yellow","red","green"]})


Let's look at our data by printing it out. Using this dataset, we will try to answer simple questions like the probability of a banana being tasty given its size.  

In [ ]:
print(data)

## Step 2. Create a Bayesian network
We will now create a Bayesian Network for our dataset. We will assume that the network will have `fruit` as the parent of `size` and `color` and, these two will determine if the fruit is tasty! We will use the `BayesianNetwork` class from pgmpy.

In [ ]:
from pgmpy.models import BayesianNetwork

Specify the structure of the Bayesian network by adding edges from parent to child.

In [ ]:
model=BayesianNetwork([('fruit','size'),('fruit','color'),('size', 'tasty'), ('color', 'tasty')])

Let's draw out the structure of the Bayesian network. We will use the `networkx` package for this.

In [ ]:
import importlib
import networkx as nx
import matplotlib.pyplot as plt
pos=nx.circular_layout(model)
#pos=nx.spring_layout(model)
#pos=nx.planar_layout(model)
nx.draw(model,pos=pos,with_labels=True,node_size=500,font_size=15,alpha=0.5);
plt.show()

## Step 3. Parameter estimation of the Bayesian network
Now we will do some count estimations based on the data we had previously generated. For this we will use the `ParameterEstimator` class of pgmpy. We will first do simple count estimations, then maximum likelihood estimates, and finally we will use a pseudo count (a simple version of adding parameter priors).

### Step 3a. Count estimations

In [ ]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)

Let's look at some count estimates of the variables. Let's do this for `fruit` and `size`. 

In [ ]:
print("\n", pe.state_counts('fruit')) 

In [ ]:
print("\n", pe.state_counts('size')) 

### Step 3b. Maximum likelihood estimation
Now, we will get Maximum Likelihood estimations of the conditional distributions. This can be done by using the `MaximumLikelihoodEstimator` of pgmpy. 

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator

mle = MaximumLikelihoodEstimator(model, data)

We can obtain the entries of the CPD for each variable using the `mle.estimate_cpd` function. Let's do it for `fruit`. CPD for `fruit`:

In [ ]:
print(mle.estimate_cpd('fruit'))

Similarly, the CPD for `size`:

In [ ]:
print(mle.estimate_cpd('size'))

### Step 3c. Adding parameter priors
The MLE of parameters can be problematic as it might not generalize well to new samples. Addition of parameter priors enables us to provide more robust estimates and also to incorporate any prior knowledge we might have about the parameters/model. Here we will consider the simplest type of parameter prior where we add a *pseudo count* of 1 to each CPD entry. This is also called the **K2** prior.

In [ ]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

As in the MLE estimates, let's examine the CPD of different variables with the K2 prior. The CPD of the `size` variable:

In [ ]:
print(est.estimate_cpd('size',prior_type='K2'))

The CPD of the `tasty` variable:

In [ ]:
tasty_cpd=est.estimate_cpd('tasty',prior_type='K2')
print(tasty_cpd)

The print function may suppress some columns to make everything fit. For large parent size, it might be better to use `get_values()` function and `variables`.

In [ ]:
print(tasty_cpd.variables)
print(tasty_cpd.get_values())